In [4]:
from pytube import YouTube
from moviepy.editor import VideoFileClip
import deepface
import cv2
import os
import re
import os
import json
import logging
from moviepy.editor import VideoFileClip
from concurrent.futures import ThreadPoolExecutor, as_completed
import shutil
import os


we unify the json first
to solve the problem we will first unify the three json into one excel or csv whichever is faster for indexing and we add aditional column named test train and validation , 

In [13]:
import pandas as pd
import json

# Paths to the JSON files
json_files = [
    r"D:\ASL\Dataset\Json\MSASL_test_with_videos.json",
    r"D:\ASL\Dataset\Json\MSASL_train_with_videos.json",
    r"D:\ASL\Dataset\Json\MSASL_val_with_videos.json"
]

# Function to load JSON and add a 'dataset' column
def load_json_with_dataset(json_file, dataset_type):
    """
    Load a JSON file and add a column to specify the dataset type (test/train/val).
    
    Parameters:
    - json_file (str): Path to the JSON file.
    - dataset_type (str): Type of dataset ('test', 'train', 'val') to be added as a column.
    
    Returns:
    - DataFrame: A pandas DataFrame with an additional 'dataset' column.
    """
    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        # Convert JSON to DataFrame
        df = pd.json_normalize(data)  # Flatten nested JSON (like 'box' list)
        
        # Add the dataset type column
        df['dataset'] = dataset_type
        return df
    except Exception as e:
        print(f"Error loading {json_file}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error

# Load all datasets and concatenate them
all_data = []
for json_file, dataset_type in zip(json_files, ['test', 'train', 'val']):
    df = load_json_with_dataset(json_file, dataset_type)
    all_data.append(df)

# Concatenate all DataFrames into one
unified_df = pd.concat(all_data, ignore_index=True)

# Save to CSV
unified_csv_path = r"D:\ASL\Dataset\unified_data.csv"
unified_df.to_csv(unified_csv_path, index=False)

print(f"Unified dataset saved to: {unified_csv_path}")


Unified dataset saved to: D:\ASL\Dataset\unified_data.csv


In [14]:

df = pd.read_csv(unified_csv_path)

# Function to extract video ID from YouTube URL
def extract_video_id(url):
    """
    Extracts the video ID from a YouTube URL.
    
    Parameters:
    - url (str): The YouTube URL.
    
    Returns:
    - str: The extracted video ID.
    """
    match = re.search(r"v=([a-zA-Z0-9_-]+)", url)
    if match:
        return match.group(1)
    else:
        return None  # Return None if no match found

# Apply the function to the 'url' column and update the 'video' column
df['video'] = df['url'].apply(extract_video_id)

# Save the updated DataFrame to a new CSV file (or overwrite the existing one)
updated_csv_path = r"D:\ASL\Dataset\updated_unified_data.csv"
df.to_csv(updated_csv_path, index=False)

print(f"Updated dataset with video IDs saved to: {updated_csv_path}")


Updated dataset with video IDs saved to: D:\ASL\Dataset\updated_unified_data.csv


we move into respective folders
    

In [ ]:
import os
import shutil
import pandas as pd

# Paths
videos_folder = r"D:\ASL\Dataset\videos"
test_folder = r"D:\ASL\Dataset\videos\test"
train_folder = r"D:\ASL\Dataset\videos\train"
val_folder = r"D:\ASL\Dataset\videos\val"
csv_path = r"D:\ASL\Dataset\updated_unified_data.csv"

# Load the updated CSV file
df = pd.read_csv(csv_path)

# Function to find the video file with any extension in the source folder
def find_video_file(video_id):
    """
    Find the video file by matching the video ID with any extension in the source folder.
    
    Parameters:
    - video_id (str): The video ID to search for.
    
    Returns:
    - str: The full path of the video file if found, otherwise None.
    """
    for filename in os.listdir(videos_folder):
        if video_id in filename:  # Check if the video_id is in the filename
            return os.path.join(videos_folder, filename)  # Return full path of the file
    return None  # Return None if no video is found

# Function to move the video file based on the video ID and dataset type
def move_video(row):
    video_id = row['video']
    dataset_type = row['dataset']
    
    # Find the video file in the source folder with the correct video ID
    source_path = find_video_file(video_id)
    
    if source_path:
        # Determine the destination folder based on the 'dataset' column
        if dataset_type == 'test':
            destination_folder = test_folder
        elif dataset_type == 'train':
            destination_folder = train_folder
        elif dataset_type == 'val':
            destination_folder = val_folder
        else:
            print(f"Invalid dataset type for video ID {video_id}: {dataset_type}")
            return
        
        # Construct the destination path
        destination_path = os.path.join(destination_folder, os.path.basename(source_path))
        
        # Check if the video exists in the source folder and move it to the destination folder
        if not os.path.exists(destination_path):  # Check if the file already exists in the destination folder
            try:
                shutil.move(source_path, destination_path)
                print(f"Moved {os.path.basename(source_path)} to {destination_folder}")
            except Exception as e:
                print(f"Error moving {os.path.basename(source_path)}: {e}")
        else:
            print(f"Video {os.path.basename(source_path)} already exists in {destination_folder}, skipping.")
    else:
        print(f"Video with ID {video_id} not found in {videos_folder}, skipping.")

# Apply the move_video function to each row of the DataFrame
df.apply(move_video, axis=1)

print("Video moving process completed.")


Corrects the YouTube URL format if necessary, downloads the video using pytube,
    and saves it to the corresponding folder based on dataset type (test, train, or val).
    
    Parameters:
    - url (str): The original URL of the YouTube video.
    - dataset_type (str): The dataset category, which determines the output folder 
                          (e.g., 'test', 'train', or 'val').

    Returns:
    - file_path (str): The path where the downloaded video is saved.

Step 3: Slicing the Video Based on Start and End Times
Now that we’ve successfully downloaded the videos, our next task is to slice these videos into segments based on the start and end times specified in the JSON files. Each segment will correspond to a specific American Sign Language (ASL) gesture.

Purpose of Video Slicing
The JSON file provides timing information for each ASL gesture within the YouTube video, allowing us to create individual video clips focusing on each gesture. This step is essential for isolating gestures for further processing, such as facial sentiment analysis and hand gesture detection.

Approach
Identify Start and End Times:
Extract the start_time and end_time values from the JSON file for each entry. These values denote the time range for each ASL gesture within the downloaded video.
Extract Video Segment:
Using the moviepy library, open the downloaded video file and extract the segment that matches the start_time and end_time values.
Save Each Segment:
Save the extracted video segment locally in the appropriate output folder (test, train, or val) based on the dataset type, with each file named by its class (clean_text).
Output Format
Each sliced video segment will be saved in the corresponding output folder (e.g., ./MSAL_test_output) with a filename corresponding to the class name of the gesture (e.g., forget.mp4 for a segment representing the "forget" gesture).

In [1]:
import os
from moviepy.editor import VideoFileClip
import pandas as pd

# Paths to folders
train_folder = r"D:\ASL\Dataset\videos\train"
test_folder = r"D:\ASL\Dataset\videos\test"
val_folder = r"D:\ASL\Dataset\videos\val"
csv_path = r"D:\ASL\Dataset\updated_unified_data.csv"

# Load the updated CSV file
df = pd.read_csv(csv_path)

# Function to slice the video based on start_time and end_time
def slice_video(row):
    video_id = row['video']
    clean_text = row['clean_text']
    start_time = row['start_time']
    end_time = row['end_time']
    dataset_type = row['dataset']
    
    # Skip if the video has already been processed
    output_file = get_output_file_path(row)
    if os.path.exists(output_file):
        print(f"Video {clean_text} already sliced, skipping.")
        return

    # Find the video file in the respective folder
    if dataset_type == 'test':
        video_folder = test_folder
    elif dataset_type == 'train':
        video_folder = train_folder
    elif dataset_type == 'val':
        video_folder = val_folder
    else:
        return
    
    # Find the full video file path (with any extension)
    video_path = find_video_file(video_id, video_folder)
    
    if video_path:
        try:
            # Load the video file
            video_clip = VideoFileClip(video_path)
            
            # Slice the video based on start_time and end_time
            sliced_clip = video_clip.subclip(start_time, end_time)
            
            # Write the sliced video to output file
            sliced_clip.write_videofile(output_file, codec="libx264", audio_codec="aac")
            
            print(f"Sliced video saved as {output_file}")
        except Exception as e:
            print(f"Error slicing video {video_id} with clean_text {clean_text}: {e}")
    else:
        return

# Function to find the video file by video_id in a given folder
def find_video_file(video_id, folder):
    """
    Find the video file by matching the video ID with any extension in the specified folder.
    
    Parameters:
    - video_id (str): The video ID to search for.
    - folder (str): The folder to search in.
    
    Returns:
    - str: The full path of the video file if found, otherwise None.
    """
    for filename in os.listdir(folder):
        if video_id in filename:  # Check if the video_id is in the filename
            return os.path.join(folder, filename)  # Return full path of the file
    return None  # Return None if no video is found

# Function to get the output file path based on clean_text and dataset type
def get_output_file_path(row):
    clean_text = row['clean_text']
    dataset_type = row['dataset']
    
    # Choose the folder based on dataset type
    if dataset_type == 'test':
        folder = test_folder
    elif dataset_type == 'train':
        folder = train_folder
    elif dataset_type == 'val':
        folder = val_folder
    else:
        return None
    
    # Construct the output file path
    return os.path.join(folder, f"{clean_text}.mp4")

# Apply the slice_video function to each row of the DataFrame
df.apply(slice_video, axis=1)

print("Video slicing process completed.")


Video bad already sliced, skipping.
Video happy already sliced, skipping.
Video cashier already sliced, skipping.
Video both already sliced, skipping.
Video mean already sliced, skipping.
Video laugh already sliced, skipping.
Video spain already sliced, skipping.
Video holland already sliced, skipping.
Video pshaw already sliced, skipping.
Video time already sliced, skipping.
Video spring already sliced, skipping.
Video good afternoon already sliced, skipping.
Video hello already sliced, skipping.
Video angry already sliced, skipping.
Video doctor already sliced, skipping.
Video game already sliced, skipping.
Video money already sliced, skipping.
Video what's up already sliced, skipping.
Video argentina already sliced, skipping.
Video time already sliced, skipping.
Video friendly already sliced, skipping.
Video motorcycle already sliced, skipping.
Video hard already sliced, skipping.
Video interpreter already sliced, skipping.
Video any already sliced, skipping.
Video forget already sl

In this step, we’ll analyze the facial expressions in each sliced video segment to determine the sentiment (e.g., happy, sad, neutral). This sentiment information will be used in the next step to rename the video files with both the class and the detected sentiment, such as "beer-happy.mp4".

Purpose of Facial Sentiment Analysis
This analysis allows us to add contextual information to each ASL gesture by associating a sentiment, which could be valuable for understanding the emotional context of each gesture in training data.

Approach
Extract Frames: Load the video segment and extract frames. We don’t need every frame for sentiment analysis, so analyzing every nth frame (e.g., every second frame) is usually sufficient.

Analyze Each Frame: For each selected frame, we’ll use a facial sentiment analysis model to detect emotions. We’ll use the DeepFace library, which provides pre-trained models for this purpose.

Determine Dominant Sentiment: Analyze the detected emotions and choose the dominant one. This could be done by calculating the most frequently detected emotion across the sampled frames.

Return Sentiment Label: The dominant emotion is returned as the overall sentiment for that video segment.

In [2]:
import os
import pandas as pd

# Paths to folders
train_folder = r"D:\ASL\Dataset\videos\train"
test_folder = r"D:\ASL\Dataset\videos\test"
val_folder = r"D:\ASL\Dataset\videos\val"
csv_path = r"D:\ASL\Dataset\updated_unified_data.csv"

# Load the updated CSV file
df = pd.read_csv(csv_path)

# Function to delete the original video files after slicing
def delete_original_files():
    for _, row in df.iterrows():
        video_id = row['video']
        clean_text = row['clean_text']
        dataset_type = row['dataset']
        
        # Determine the folder based on the dataset type
        if dataset_type == 'test':
            video_folder = test_folder
        elif dataset_type == 'train':
            video_folder = train_folder
        elif dataset_type == 'val':
            video_folder = val_folder
        else:
            continue
        
        # Find the full video file path (with any extension)
        video_path = find_video_file(video_id, video_folder)
        
        # Check if the sliced video exists
        output_file = get_output_file_path(row)
        
        if os.path.exists(output_file) and video_path:
            try:
                # Delete the original video file
                os.remove(video_path)
                print(f"Original video {video_id} deleted.")
            except Exception as e:
                print(f"Error deleting video {video_id}: {e}")
        else:
            print(f"Video {video_id} or sliced file not found, skipping deletion.")

# Function to find the video file by video_id in a given folder
def find_video_file(video_id, folder):
    """
    Find the video file by matching the video ID with any extension in the specified folder.
    
    Parameters:
    - video_id (str): The video ID to search for.
    - folder (str): The folder to search in.
    
    Returns:
    - str: The full path of the video file if found, otherwise None.
    """
    for filename in os.listdir(folder):
        if video_id in filename:  # Check if the video_id is in the filename
            return os.path.join(folder, filename)  # Return full path of the file
    return None  # Return None if no video is found

# Function to get the output file path based on clean_text and dataset type
def get_output_file_path(row):
    clean_text = row['clean_text']
    dataset_type = row['dataset']
    
    # Choose the folder based on dataset type
    if dataset_type == 'test':
        folder = test_folder
    elif dataset_type == 'train':
        folder = train_folder
    elif dataset_type == 'val':
        folder = val_folder
    else:
        return None
    
    # Construct the output file path
    return os.path.join(folder, f"{clean_text}.mp4")

# Call the delete_original_files function to delete already processed files
delete_original_files()

print("Original files deletion process completed.")


Video wX78EPtSuzU or sliced file not found, skipping deletion.
Video OL02Odh2dRg or sliced file not found, skipping deletion.
Video C59jcSo4fEI or sliced file not found, skipping deletion.
Video Qs2ua1S6tg0 or sliced file not found, skipping deletion.
Video -kgTBeOw95A or sliced file not found, skipping deletion.
Video ieLzL1Sl6SU or sliced file not found, skipping deletion.
Video xiJYYCuA2UA or sliced file not found, skipping deletion.
Video qSOspusLa3U or sliced file not found, skipping deletion.
Video 78bHeANdyYc or sliced file not found, skipping deletion.
Video deGERv54kIc or sliced file not found, skipping deletion.
Video ZcFkE7qjmcc or sliced file not found, skipping deletion.
Video a1NeXe8bhuo or sliced file not found, skipping deletion.
Video 2vYnEG9tSuA or sliced file not found, skipping deletion.
Video e22Pc68i_VE or sliced file not found, skipping deletion.
Video OdJuIwj6iYY or sliced file not found, skipping deletion.
Video BBCixbI_6C4 or sliced file not found, skipping de

In [4]:
pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.7 MB 1.1 MB/s eta 0:00:02
   ---------------------------------------  1.7/1.7 MB 15.7 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 13.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import pandas as pd

# Paths to folders
train_folder = r"D:\ASL\Dataset\videos\train"
test_folder = r"D:\ASL\Dataset\videos\test"
val_folder = r"D:\ASL\Dataset\videos\val"
csv_path = r"D:\ASL\Dataset\updated_unified_data.csv"

# Load the updated CSV file
df = pd.read_csv(csv_path)

# Get the list of clean_text entries (these should be the video names)
clean_text_list = df['clean_text'].tolist()

# Function to get the video filenames from the folder
def get_video_files(folder):
    """
    Returns a list of video filenames (without extensions) from the specified folder.
    """
    video_files = []
    for filename in os.listdir(folder):
        if filename.endswith(('.mp4', '.avi', '.mov', '.mkv')):  # Add more extensions if needed
            video_files.append(os.path.splitext(filename)[0])  # Get the name without the extension
    return video_files

# Get the list of video files from each folder
train_videos = get_video_files(train_folder)
test_videos = get_video_files(test_folder)
val_videos = get_video_files(val_folder)

# Combine the lists of available videos
available_videos = set(train_videos + test_videos + val_videos)

# Find the missing videos
missing_videos = [video for video in clean_text_list if video not in available_videos]

# Filter rows from the original CSV corresponding to the missing videos
missing_rows = df[df['clean_text'].isin(missing_videos)]

# Save the rows with missing videos to a new CSV file
missing_rows_csv_path = r"D:\ASL\Dataset\missing_videos.csv"
missing_rows.to_csv(missing_rows_csv_path, index=False)

print(f"New CSV with missing videos saved at {missing_rows_csv_path}.")


New CSV with missing videos saved at D:\ASL\Dataset\missing_videos.csv.


Next Step: Renaming the File with Class and Sentiment
Now that we have the dominant sentiment for each video segment, the next step is to rename the segment files to include both the class and sentiment, following the format class-sentiment.mp4 (e.g., beer-happy.mp4).

Approach
Define File Naming Convention:
Use the gesture class (clean_text) and the detected sentiment to create the filename in the format class-sentiment.mp4.
Rename the File:
Save the renamed file in the appropriate output directory (test, train, or val), replacing the original filename.

In [10]:
import mediapipe as mp
import cv2

def extract_hand_gesture(video_path, label, sentiment, dataset_type):
    """
    Extracts hand gesture landmarks from a video segment, creates a stick-man
    representation, and saves it in a simplified format to reduce storage space.
    
    Parameters:
    - video_path (str): Path to the video segment file.
    - label (str): Class name for the gesture.
    - sentiment (str): Detected sentiment for the video segment.
    - dataset_type (str): The dataset category (e.g., 'test', 'train', or 'val').
    
    Returns:
    - output_path (str): Path where the processed hand gesture video is saved.
    """
    # Define MediaPipe Hands solution
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    
    # Set up OpenCV video capture for reading frames from the original video
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define output file path with the naming convention
    output_name = f"{label}-{sentiment}.mp4"
    output_path = os.path.join(BASE_PATHS[dataset_type], output_name)
    
    # Set up video writer for the stick-man output
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    # Process each frame
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # Convert the frame to RGB as required by MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        
        # Draw landmarks if hands are detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks and connections on the frame
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Write the processed frame to the output video
        out.write(frame)

    # Release resources
    cap.release()
    out.release()
    hands.close()
    print(f"Hand gesture video saved at: {output_path}")
    
    return output_path


Pipeline

pip install opencv-python opencv-python-headless numpy


In [5]:
import cv2
import numpy as np
import random

def rotate_frame(frame, angle):
    """
    Rotates a frame by a specific angle.
    """
    (h, w) = frame.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_frame = cv2.warpAffine(frame, rotation_matrix, (w, h))
    return rotated_frame

def flip_frame(frame, flip_type=1):
    """
    Flip a frame. flip_type=1 is horizontal, flip_type=0 is vertical.
    """
    return cv2.flip(frame, flip_type)

def scale_frame(frame, scale_factor=1.2):
    """
    Scales a frame by a certain factor.
    """
    width = int(frame.shape[1] * scale_factor)
    height = int(frame.shape[0] * scale_factor)
    return cv2.resize(frame, (width, height))

def add_noise_to_frame(frame):
    """
    Adds random noise to the frame.
    """
    row, col, ch = frame.shape
    mean = 0
    sigma = 25
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy = cv2.add(frame, gauss.astype('uint8'))
    return noisy

def data_augmentation(video_path, num_augmentations=5):
    """
    Perform augmentation on the input video and save the augmented versions.
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    augmented_paths = []
    
    for i in range(num_augmentations):
        # Define output file path
        augmented_name = f"augmented_{i + 1}_{video_path.split('/')[-1]}"
        output_path = video_path.replace('.mp4', f'_{i+1}.mp4')
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  # End if no more frames to read
            
            # Randomly apply augmentation
            augment_type = random.choice(['rotate', 'flip', 'scale', 'noise'])
            
            if augment_type == 'rotate':
                angle = random.randint(-30, 30)  # Random angle between -30 and 30 degrees
                frame = rotate_frame(frame, angle)
                
            elif augment_type == 'flip':
                flip_type = random.choice([0, 1])  # 0 for vertical flip, 1 for horizontal
                frame = flip_frame(frame, flip_type)
                
            elif augment_type == 'scale':
                scale_factor = random.uniform(0.8, 1.5)  # Random scale factor between 0.8 and 1.5
                frame = scale_frame(frame, scale_factor)
                
            elif augment_type == 'noise':
                frame = add_noise_to_frame(frame)
            
            # Write the augmented frame to the output video
            out.write(frame)
        
        # Close the video writer and add the path to augmented video to the list
        out.release()
        augmented_paths.append(output_path)
        
        # Reset the video capture object for the next iteration
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    # Release the video capture object
    cap.release()
    
    return augmented_paths


pip install opencv-python opencv-python-headless numpy


In [8]:
def process_single_entry(entry, dataset_type):
    """
    Processes a single entry in the ASL dataset (slicing, sentiment analysis, etc.).
    Assumes the video is already downloaded in the local path.
    
    Parameters:
    - entry: A single entry from the JSON dataset.
    - dataset_type: The type of the dataset ('train', 'test', 'val').
    """
    try:
        # Step 1: Get the video path based on the ID from the entry
        video_id = entry.get("id")  # or whatever field contains the video ID
        video_path = get_video_path_from_local(video_id, dataset_type)
        if not video_path:
            raise Exception(f"Video not found for ID: {video_id}")
        
        # Step 2: Slice the video based on start and end times from the dataset
        start_time = entry.get("start_time")
        end_time = entry.get("end_time")
        label = entry.get("clean_text")  # assuming clean_text contains the gesture label
        segment_path = slice_video(video_path, start_time, end_time, label, dataset_type)
        if not segment_path:
            raise Exception(f"Failed to slice video for label: {label}")
        
        # Step 3: Perform facial sentiment analysis
        sentiment = facial_sentiment_analysis(segment_path)

        # Step 4: Rename video file with class and sentiment
        renamed_segment_path = rename_with_sentiment(segment_path, label, sentiment, dataset_type)

        # Step 5: Extract hand gesture and save as stick-man representation
        hand_gesture_path = extract_hand_gesture(renamed_segment_path, label, sentiment, dataset_type)

        # Log successful processing
        print(f"Processed '{label}' with sentiment '{sentiment}' saved at {hand_gesture_path}")

    except Exception as e:
        logging.error(f"Error processing entry '{entry.get('clean_text')}' in dataset '{dataset_type}': {str(e)}")


In [9]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
import os

def process_asl_dataset_parallel(json_file_path, dataset_type):
    """
    Processes each entry in the ASL dataset JSON file in parallel using ThreadPoolExecutor,
    and logs any errors for easy debugging.

    Parameters:
    - json_file_path (str): Path to the JSON file containing ASL dataset entries.
    - dataset_type (str): Dataset type ('test', 'train', or 'val') used to determine the output folder.
    """
    # Verify the JSON file path
    if not os.path.isfile(json_file_path):
        logging.error(f"JSON file not found: {json_file_path}")
        print(f"Error: JSON file not found at path '{json_file_path}'")
        return  # Exit function if the file is missing

    # Load the JSON data
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # Process each entry in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Submit all entries for processing
        futures = {executor.submit(process_single_entry, entry, dataset_type): entry for entry in data}
        
        # Wait for each future to complete and handle exceptions
        for future in as_completed(futures):
            try:
                # .result() will raise any exceptions caught by the future
                future.result()
            except Exception as e:
                # Log unexpected exceptions from the thread pool
                logging.error(f"Unexpected error in processing entry: {str(e)}")



In [13]:
# Example usage
setup_output_folders()  # Ensure output folders are ready

# Correct paths with .json extension
process_asl_dataset_parallel("/mnt/d/ASL/Dataset/Json/MSASL_test.json", "test")
process_asl_dataset_parallel("/mnt/d/ASL/Dataset/Json/MSASL_train.json", "train")
process_asl_dataset_parallel("/mnt/d/ASL/Dataset/Json/MSASL_val.json", "val")


Directory already exists: /mnt/d/ASL/Dataset/videos/test
Directory already exists: /mnt/d/ASL/Dataset/videos/train
Directory already exists: /mnt/d/ASL/Dataset/videos/val


In [3]:
pip install ffmpeg-python


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
